<a href="https://colab.research.google.com/github/rezamohammadloo/Surface_anomaly_detection_using_unsupervised_algorithms/blob/main/IsolationForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import os
import numpy as np
import tensorflow as tf
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, average_precision_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV



In [7]:
# Define a function to load images from a specified directory and resize them to the target size
def load_images_from_directory(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img = load_img(os.path.join(directory, filename), target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
    return np.array(images)

# Directories
# Specify the directories containing normal, anomaly, and mask images
normal_dir = '/content/gdrive/MyDrive/glass-defect-sample/good'
anomaly_dir = '/content/gdrive/MyDrive/glass-defect-sample/anomaly'
mask_dir = '/content/gdrive/MyDrive/glass-defect-sample/mask'

# Load data
# Load the normal, anomaly, and mask images from their respective directories
normal_images = load_images_from_directory(normal_dir)
anomaly_images = load_images_from_directory(anomaly_dir)
masks = load_images_from_directory(mask_dir)

In [9]:
# Flatten images to vectors
def flatten_images(images):
    return images.reshape(images.shape[0], -1)

X_good = flatten_images(normal_images)
X_anomaly = flatten_images(anomaly_images)




In [10]:
# Scale the data
scaler = StandardScaler()
X_good = scaler.fit_transform(X_good)
X_anomaly = scaler.transform(X_anomaly)


In [12]:
# Define the parameter grid

# Train Isolation Forest
isolation_forest = IsolationForest( contamination='auto', random_state=42)
isolation_forest.fit(X_good)

IsolationForest(random_state=42)

In [13]:
all_images = np.concatenate((X_good, X_anomaly), axis=0)
# all_labels = np.concatenate((np.zeros(X_good.shape[0]), np.ones(X_anomaly.shape[0])))
all_labels = np.concatenate((np.ones(X_good.shape[0]) , -1*np.ones(X_anomaly.shape[0])))
predictions = isolation_forest.predict(all_images)


In [14]:
if_aucroc = roc_auc_score(all_labels, predictions)
print(f"AUC-ROC for Isolation Forest: {if_aucroc:0.04f}" )

if_ap = average_precision_score(all_labels, predictions)
print(f"AP for Isolation Forest: {if_ap:0.04f}")

AUC-ROC for Isolation Forest: 0.8363
AP for Isolation Forest: 0.9260
